In [1]:
import tensorflow as tf
import modin.pandas as pd
import numpy as np
import pickle

import ray
ray.init(ignore_reinit_error=True)

E0414 08:45:30.701370874   15714 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0414 08:45:30.724962264   15714 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0414 08:45:30.746957305   15714 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


{'node_ip_address': '172.17.0.2',
 'raylet_ip_address': '172.17.0.2',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-04-14_08-45-29_630876_15714/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-04-14_08-45-29_630876_15714/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-04-14_08-45-29_630876_15714',
 'metrics_export_port': 56165,
 'gcs_address': '172.17.0.2:59219',
 'address': '172.17.0.2:59219',
 'node_id': '60ec33b76386a20b056970fa50adc52c9fddd793431335c141872163'}

In [2]:
# Read in data
def data():

    articles_df_original = pd.read_csv("../h-and-m-recommender-system/articles.csv")
    customers_df_original = pd.read_csv("../h-and-m-recommender-system/customers.csv")
    transactions_df_original = pd.read_csv("../h-and-m-recommender-system/transactions_train.csv")

    return articles_df_original, customers_df_original, transactions_df_original

articles_df_original, customers_df_original, transactions_df_original = data()

In [3]:
# Create transactions matrix
transactions_matrix = transactions_df_original[["customer_id","article_id"]]

# Sort by customer_id
transactions_matrix = transactions_matrix.sort_values(by=["customer_id"])

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [4]:
# Split data into separate dataframes
n = 200000
list_df_transactions = [transactions_matrix[i:i+n] for i in range(0,transactions_matrix.shape[0],n)]

# Get quantity of items purchased by each user
list_df_transactions_quantity = []

for df in list_df_transactions:
    list_df_transactions_quantity.append(df.groupby(["customer_id","article_id"]).size().reset_index())

In [5]:
# Concatenate split dfs
df_transactions_quantity_original = pd.concat(list_df_transactions_quantity)
df_transactions_quantity_original = df_transactions_quantity_original.rename(columns={0:"quantity"})

In [6]:
# Change dtype to object
df_transactions_quantity_original['article_id'] = df_transactions_quantity_original['article_id'].astype('string')
df_transactions_quantity_original['quantity'] = df_transactions_quantity_original['quantity'].astype('float32')

In [7]:
df_transactions_quantity_original

,customer_id,article_id,quantity
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023,1.0
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006,2.0
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043,1.0
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,607642008,1.0
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001,1.0
...,...,...,...
161476,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,821395003,1.0
161477,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,840360003,1.0
161478,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,866755002,2.0
161479,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,882810001,1.0


In [8]:
with open('df_transactions_article_id.pkl','wb') as handle:
    pickle.dump(df_transactions_quantity_original, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
transactions_dict = {name: np.array(value) for name, value in df_transactions_quantity_original.items()}

In [10]:
with open('df_transactions_article_id.pkl','rb') as handle:
    transactions_dict = pickle.load(handle)

In [13]:
# Get numpy array of quantity values
quantity_variable = transactions_dict.get("quantity")

# Set all values to float
quantity_variable = np.asarray(quantity_variable).astype("float")

# Update quantity values in dict
transactions_dict.update(quantity=quantity_variable)

In [14]:
# Get numpy array of product code values
article_id_variable = transactions_dict.get("article_id")

# Set all values to string
article_id_variable = np.asarray(article_id_variable).astype("str")

# Update quantity values in dict
transactions_dict.update(article_id=article_id_variable)

In [15]:
transactions_dict

{'customer_id': array(['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
        '00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
        '00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
        ...,
        'ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38b2236865d949d4df6a',
        'ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38b2236865d949d4df6a',
        'ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241'],
       dtype=object),
 'article_id': array(['176209023', '568601006', '568601043', ..., '866755002',
        '882810001', '806050001'], dtype='<U9'),
 'quantity': array([1., 2., 1., ..., 2., 1., 1.])}

In [16]:
type(transactions_dict.get("article_id")[0])

numpy.str_

In [17]:
transactions = tf.data.Dataset.from_tensor_slices(transactions_dict)

2022-04-14 08:44:41.302311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-14 08:44:41.367011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-14 08:44:41.367310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-14 08:44:41.396533: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [18]:
len(transactions)

27306843

In [19]:
transactions

<TensorSliceDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'article_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'quantity': TensorSpec(shape=(), dtype=tf.float64, name=None)}>

In [ ]:
# Next step here is to use TFRecord

In [ ]:
tf.data.experimental.save(transactions, "tf_transactions_article_id")

In [2]:
tf_transactions = tf.data.experimental.load("tf_transactions")

2022-04-13 04:11:51.705866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-13 04:11:51.767320: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-13 04:11:51.767535: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-13 04:11:51.768430: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [14]:
tf_transactions

<_LoadDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'quantity': TensorSpec(shape=(), dtype=tf.float64, name=None), 'product_code': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
# Load dataframe

In [4]:
with open('df_transactions_quantity_original.pkl','rb') as handle:
    df_transactions_quantity_original = pickle.load(handle)

In [ ]:
# Fails at this point everytime

In [15]:
items_dict = df_transactions_quantity_original['product_code']

In [16]:
items_dict

0         176209
1         568601
2         607642
3         625548
4         627759
           ...  
144298    821395
144299    840360
144300    866755
144301    882810
144302    806050
Name: product_code, Length: 24415302, dtype: string

In [17]:
items_dict = items_dict.reset_index(drop=True).drop_duplicates()

In [18]:
items_dict = items_dict.reset_index(drop=True)

In [25]:
np.array(items_dict)

array(['176209', '568601', '607642', ..., '618853', '533932', '568499'],
      dtype=object)

In [26]:
items_dict_tf = {"product_code": np.array(items_dict)}

In [27]:
items_dict_tf

{'product_code': array(['176209', '568601', '607642', ..., '618853', '533932', '568499'],
       dtype=object)}

In [28]:
tf_items = tf.data.Dataset.from_tensor_slices(items_dict_tf)

In [29]:
tf_items

<TensorSliceDataset element_spec={'product_code': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [30]:
tf.data.experimental.save(tf_items, "tf_items")

In [3]:
tf_items = tf.data.experimental.load("tf_items")

In [4]:
tf_transactions = tf_transactions.map(lambda x: {'customer_id' : x['customer_id'], 
                                            'product_code' : x['product_code'], 
                                            'quantity' : float(x['quantity']),})

In [9]:
len(tf_transactions)

24415302

In [6]:
tf_items = tf_items.map(lambda x: x['product_code'])

In [8]:
len(tf_items)

46834